<a href="https://colab.research.google.com/github/neerajhon/project/blob/main/Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'new-plant-diseases-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F78313%2F182633%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240427%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240427T042825Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D518137c5b84e97fd9eb74b9b5021ba3059405b29367bb201d71a13ae70cbad6384ef6d6d70b2d56ba1d944496730c3391f20624c32e8cad033a175c9fb6c8a83f95b3dcb5c7ce5169e082b6d305ad8dd7e117d49b548f80dabbb71489a2fe6efa93d980074e3927bcb85cc872610dc2e46541231455872df7ad375c06882e30f4b9665374675f7d99452a1a52f1d24fb9f0fd792b12bd2f37449be24bc824e6d58dd34470489e7519ac71590e3c1f9aec0e58341511844fd05ba91cc4ca92b5d5ee2e416744dfa663a2a8a41ef77882c2fe3a3af150325035d4adc8d4b9dbf42f8ee8d9c374620926c257a6e40c4db77335d12d6a3f89f455e2475980c80b0ce'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import numpy as np
from keras.applications import ResNet50
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
# Setting up the directories for train, validation, and test data
train_dir = '../input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'
valid_dir = '../input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'
test_dir = '../input/new-plant-diseases-dataset/test'

In [ ]:
# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=1,
    class_mode='categorical',
    shuffle=False
)

In [ ]:
# Define the ResNet50 model for feature extraction
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
# Adding custom layers on top of ResNet50
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.0001)))
model.add(Dropout(0.5))  # Adding dropout for regularization
model.add(Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.0001)))
model.add(Dense(38, activation='softmax'))

In [ ]:
# Defining the optimizer with learning rate decay and early stopping
optimizer = Adam(learning_rate=0.001, clipnorm=1.0)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [ ]:
# Compiling the model
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Training the model with callbacks for learning rate scheduling and early stopping
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    epochs=10,  # Increase the number of epochs
    validation_data=valid_generator,
    validation_steps=valid_generator.n // valid_generator.batch_size,
    callbacks=[reduce_lr, early_stopping]
)


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

# Load a single test image
img_path = '/kaggle/input/new-plant-diseases-dataset/test/test/TomatoYellowCurlVirus3.JPG'  # Replace 'path_to_your_single_test_image.jpg' with the actual path to your image
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# Preprocess the image
img_array = img_array / 255.  # Normalize the image

# Predict the class probabilities
predictions = model.predict(img_array)

# Get the predicted class label
predicted_class = np.argmax(predictions)

# Define a dictionary mapping class indices to class labels
class_labels = train_generator.class_indices

# Inverse mapping to get the class label from the index
predicted_label = [k for k, v in class_labels.items() if v == predicted_class][0]

# Check if the predicted label contains the word "healthy"
is_diseased = "healthy" not in predicted_label.lower()

# Determine the final prediction
prediction_result = "Diseased" if is_diseased else "Non-Diseased"

# Display the image
plt.imshow(img)
plt.axis('off')
plt.title(f'Original Label: {img_path.split("/")[-1]}\nPredicted Label: {predicted_label}\nPrediction Result: {prediction_result}')
plt.show()


In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

# Directory containing all test images
test_dir = '/kaggle/input/new-plant-diseases-dataset/test/test'  # Replace with the actual path to your test directory

# Get all image file names in the test directory
test_image_files = os.listdir(test_dir)

# Loop over each image file
for img_file in test_image_files:
    # Load the image
    img_path = os.path.join(test_dir, img_file)
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    # Preprocess the image
    img_array = img_array / 255.  # Normalize the image

    # Predict the class probabilities
    predictions = model.predict(img_array)

    # Get the predicted class label
    predicted_class = np.argmax(predictions)

    # Define a dictionary mapping class indices to class labels
    class_labels = train_generator.class_indices

    # Inverse mapping to get the class label from the index
    predicted_label = [k for k, v in class_labels.items() if v == predicted_class][0]

    # Check if the predicted label contains the word "healthy"
    is_diseased = "healthy" not in predicted_label.lower()

    # Determine the final prediction
    prediction_result = "Diseased" if is_diseased else "Non-Diseased"

    # Display the image along with prediction result
    plt.imshow(img)
    plt.axis('off')
    plt.title(f'Original Label: {img_file}\nPredicted Label: {predicted_label}\nPrediction Result: {prediction_result}')
    plt.show()
